In [3]:
from scipy.optimize import linprog
import numpy as np
import pickle

### First define some utility pairings based on a scenario

In [4]:
def generate_utility_pairings(I,T):
    '''Create a dummy scenario statisfying valid weight properties'''
    impatient = np.random.rand(T,I+1)
    patient = np.random.rand(I,I+1)
    
    patient[:,0] = 0
    #make a few paris non-compatible
    pairing_weights = np.concatenate((impatient, patient), axis=0)
    for i in range(I+T):
        i,j = np.random.randint(0,I+T), np.random.randint(1,I+1)
        pairing_weights[i,j] = 0
    
    #make patient-patient pairings symmetric
    pairing_weights[T:,1:] = np.maximum(pairing_weights[T:,1:], pairing_weights[T:,1:].T)
    pairing_weights[T:,1:][np.eye(I,dtype=bool)] = 0
    
    return pairing_weights

In [46]:
T = 10 #impatient agents
I = 10 #patient agents

pairing_weights = generate_utility_pairings(I,T)

### Now define several functions to convert our problem formulation to standard matrix form

In [6]:
def make_primal_constraint_matrix(I, T, c_len, pairing_weights):
    constraints = np.zeros((T+I, c_len))
    
    for a in range(0, T):
        constraint = np.zeros(pairing_weights.shape)
        constraint[a, :] = 1
        constraints[a,:] = constraint.reshape(constraint.shape[0] * constraint.shape[1])
    
    for B in range(0, I):
        constraint = np.zeros(pairing_weights.shape)
        constraint[B+T, 1:] += 1
        constraint[:, B+1] += 1
        constraints[B+T,:] = constraint.reshape(constraint.shape[0] * constraint.shape[1])
    
    return constraints

def make_dual_constraint_matrix(I, T, pairing_weights):
    constraints = np.zeros(((I+1)*T+I*I, T+I))
    inequalities = np.zeros(((I+1)*T+I*I,1))
    cix = 0
    
    #impatient constraints
    for t in range(T):    
        bixs = [i for i in range(T,T+I)]
        bixs.insert(0,t)
        
        for ix, i in enumerate(bixs): 
            constraints[cix, t] = -1
            constraints[cix,i] = -1
            inequalities[cix] = -1 * pairing_weights[t, ix]
            cix += 1
            
    #patient constraints
    for i in range(I):    
        for j in range(I):
            constraints[cix, T+i] -= 1
            constraints[cix, T+j] -= 1
            inequalities[cix] = -1 * pairing_weights[T+i, j+1]
            cix += 1

    return constraints, inequalities

### Find optimal pairings & utility based on the primal. Find optimal coefficients & utility based on the dual

In [42]:
def calculate_primal_solns(pairing_weights, I, T):

    c = -1 * pairing_weights.reshape(pairing_weights.shape[0] * pairing_weights.shape[1])
    constraints = make_primal_constraint_matrix(I, T, len(c), pairing_weights)
    b = np.ones((constraints.shape[0],1))

    solved_program = linprog(c, constraints, b)
    pairings = solved_program.x.reshape(pairing_weights.shape)
    utility_p = -1 * solved_program.fun
    
    
    return utility_p, pairings

def calculate_dual_solns(pairing_weights, I, T):

    constraints_d, inequalities_d = make_dual_constraint_matrix(I, T, pairing_weights)
    c_d = np.ones((T+I))

    solved_program = linprog(c_d, constraints_d, inequalities_d)
    print(solved_program)
    coefficients = solved_program.x
    utility_d = solved_program.fun
    
    return utility_d, coefficients

### Assert utility found in primal & dual are equivalent (to precision error)

In [48]:
utility_p, pairings = calculate_primal_solns(pairing_weights, I, T)
utility_d, coeffs = calculate_dual_solns(pairing_weights, I, T)

print(utility_d)
print(utility_p)

# assert round(utility_d, 10) == round(utility_p, 10), "Dual and Primal solutions are not equivilant"

     fun: 10.156330087345138
 message: 'Optimization terminated successfully.'
     nit: 276
   slack: array([0.74581022, 1.3866515 , 0.98198989, 0.99783559, 0.23111465,
       1.14851683, 0.68870727, 0.70739428, 1.13230773, 0.73627011,
       0.62088993, 1.06427692, 1.34217012, 0.798204  , 0.70738925,
       1.02857139, 0.99311257, 0.20157708, 0.92388799, 0.87738848,
       0.        , 0.1190451 , 0.        , 0.88695654, 1.10665317,
       0.64332084, 0.        , 0.76284646, 1.25377015, 0.53928616,
       1.1603717 , 1.4045379 , 0.45797996, 0.        , 0.62044033,
       0.39639654, 0.8987182 , 0.47689605, 0.64787767, 0.44533333,
       0.77142324, 0.12304224, 0.33533224, 0.68894013, 0.        ,
       0.30322234, 0.16654868, 0.44442027, 0.        , 0.67700678,
       0.20935279, 0.0713263 , 0.32733402, 0.10430402, 0.04333372,
       0.        , 0.47535407, 0.09039977, 0.08990133, 0.84817897,
       0.56944956, 0.        , 0.2603143 , 0.61063596, 0.42974148,
       0.38114216, 0.     

In [41]:
print(utility_d)
print(utility_p)

925.8039793675945
43.66967465020719


# Testing
- Iterate over different patient/impatient scenarios and validate primal & dual produce same utility

In [121]:
for R in range(1,10):
    for S in range(1,10):
        pairing_weights = generate_utility_pairings(R,S)
        utility_1, _ = calculate_primal_solns(pairing_weights, R, S)
        utility_2, _ = calculate_dual_solns(pairing_weights, R, S)
        assert round(utility_1, 10) == round(utility_2, 10), "Dual and Primal solutions are not equivilant"

### Implement online assignment algorithm and check solution

In [98]:
def check_opt_io(shaddow_pairings, j, weights, betas):
    for ix,x in enumerate(shaddow_pairings):
        for iy,y in enumerate(shaddow_pairings):
            if x==y and x != 0 and x != -1:
                
                print(weights[ix] - betas[j] - betas[ix] == 0)
                print(weights[iy] - betas[j] - betas[iy] ==0)
                return

def online_dual_assignment(betas, pairing_weights, I, T):
    betas = np.insert(betas, 0,0)
    II = [i for i in range(I+1)]
    II_membership = np.ones((I+1), np.bool)
    matches = []
    
    for t in range(0,T):
        shaddow_pairings = pairing_weights[t, :] - betas
        shaddow_pairings[~II_membership] = -1
        io = np.argmax(shaddow_pairings)
        if io == 0:
            matches.append((t,0))
        else:
            matches.append((t, io))
            II_membership[io] = 0
            
    for j in range(1,I+1):
        #recently added -1 index. Carefully walk through to verify correct
        if II_membership[j-1]:
            II_membership_tmp = II_membership.copy()
            II_membership_tmp[j-1] = 0

            shaddow_pairings = pairing_weights[T+j-1, :] - betas
            shaddow_pairings[~II_membership_tmp] = -1
            
            
            check_opt_io(shaddow_pairings, j, pairing_weights[T+j-1, :], betas)
            io = np.argmax(shaddow_pairings)

            if io != 0:
                matches.append((T+j-1, io))
                II_membership[io] = 0
         
    pairings = np.zeros(pairing_weights.shape)
    utility = 0
    for match in matches:
        pairings[match[0], match[1]] = 1
        utility += pairing_weights[match[0], match[1]]
                
    return utility, pairings
    

In [101]:
T = 4 #impatient agents
I = 4 #patient agents


pairing_weights = generate_utility_pairings(I,T)

In [81]:
pairing_weights

array([[0.7042036 , 0.06201134, 0.18783841, 0.87904628, 0.17920747],
       [0.02106154, 0.99730342, 0.3402794 , 0.78420886, 0.        ],
       [0.81110563, 0.60102404, 0.        , 0.16127142, 0.37948676],
       [0.00509338, 0.        , 0.12046811, 0.80514355, 0.37716493],
       [0.        , 0.        , 0.        , 0.26798454, 0.77234224],
       [0.        , 0.        , 0.        , 0.37445721, 0.67586622],
       [0.        , 0.26798454, 0.37445721, 0.        , 0.66750402],
       [0.        , 0.77234224, 0.67586622, 0.66750402, 0.        ]])

In [102]:
#Get knowlege of optimal Bis
up, pairings_actual = calculate_primal_solns(pairing_weights, I, T)
ud, coeffs = calculate_dual_solns(pairing_weights, I, T)

utility, pairings_test = online_dual_assignment(coeffs[T:], pairing_weights, I, T)

print('Optimal allocations from primal')
print(pairings_actual)
print('Online allocations')
print(pairings_test)

print('\nBase utility')
print(up)
print(ud)
print('Online utility')
print(utility)

print(up - ud)

Index:  4
J:  1
[ True  True False  True  True]
[[0.42670346 0.66743212 0.         0.02540482 0.52994287]
 [0.34014008 0.         0.78154605 0.51049536 0.        ]
 [0.15546209 0.52959146 0.46950067 0.5113193  0.        ]
 [0.90259212 0.         0.84167531 0.         0.07603772]
 [0.         0.         0.72491089 0.97144976 0.42717077]
 [0.         0.72491089 0.         0.75611977 0.32071576]
 [0.         0.97144976 0.75611977 0.         0.47585698]
 [0.         0.42717077 0.32071576 0.47585698 0.        ]]
Weights:
[0.         0.         0.72491089 0.97144976 0.42717077]
Weights - betas
[-1.         -0.52936858 -1.          0.52936858  0.32393135]
False
False
+++++++++
Index:  5
J:  2
[ True  True False False  True]
[[0.42670346 0.66743212 0.         0.02540482 0.52994287]
 [0.34014008 0.         0.78154605 0.51049536 0.        ]
 [0.15546209 0.52959146 0.46950067 0.5113193  0.        ]
 [0.90259212 0.         0.84167531 0.         0.07603772]
 [0.         0.         0.72491089 0.9714